In [ ]:
import xml.etree.ElementTree as ET
from typing import Dict

import requests

from secrets.config import config

In [ ]:
def get_token(login_url: str='http://commerce.reuters.com/rmd/rest/xml/login?username={username}&password={password}',
              config: Dict[str, str] = None) -> str:
    login_url = login_url.format(username=config['user'], password=config['password'])
    try:
        response = requests.get(login_url)
        if response.ok:
            root = ET.fromstring(response.content)
            return root.text
        else:
            raise Exception('invalid response code', response)
    except Exception as e:
        raise e

auth_token = get_token(config=config)